## 我們想打造的程式

一個能夠自動將每一天的股價資料寫入到 Google SpreadSheet 的自動化程式

1. 使用者可以將想記錄股票代號填入**追蹤清單**
2. 系統會根據使用者填入的股票代號，把股價資料填入相對應的工作表内

![](https://drive.google.com/uc?export=download&id=1b8_RFRu03_-YlHFTTm_xa8qPdX2vukCE)

在上一堂課雖然我們已經寫出了能夠把股價資料寫入 Google SpreadSheet 的程式

但是若我們是將此程式放在 Colab 上，每天要手動執行十分麻煩...

因此，我們希望能夠有一個平台，能夠讓我們：

1. 將我們寫好的**程式碼部署在雲端上**，這樣就不必受限于個人電腦開關機的限制，任何時候都可以執行
2. 能夠**定期定時去執行我們寫好的程式**，免去在個人電腦上設定排程的麻煩

In [0]:
!pip install --upgrade --quiet pygsheets

## 如何讓 pygsheet 操作你的 google 試算表

我們在上一堂課有體驗到，今天要透過程式化的方式去操作 Google SpreadSheet，但是在驗證上的處理十分麻煩，需要透過手動的方式輸入驗證碼

若不希望用手動的方式輸入驗證碼，`pygsheets` 套件其實提供了一些驗證方式：

pygsheet 官方教學 (英)：[連結](https://pygsheets.readthedocs.io/en/stable/authorization.html)

---
其中最簡易的方式為，我們需要與 Google 申請一個**服務帳戶金鑰**

Google 服務帳戶金鑰官方文件 ：[連結](https://cloud.google.com/iam/docs/service-accounts)

---
### 1. 請先登入：Google 開發者頁面：[連結](https://console.developers.google.com/)

### 2. 搜尋我們在上一個教學建立的專案

![](https://drive.google.com/uc?export=download&id=1IUs8jhGkBWdw765z-sGFxrqRbDYV1Es2)

![](https://drive.google.com/uc?export=download&id=1kSlPWjbv-no6P9ZTXjPD2epRUkwmcHPW)

### 3. 點入資料庫 > 搜尋 **Sheet** 與 **Drive API**，開啓兩者

![](https://drive.google.com/uc?export=download&id=1fsWVfzMXvIGPFpoe-W2YAoi3DDu9R0jT)

![](https://drive.google.com/uc?export=download&id=1JsJPv73-dSShLFiY6E8bDOiALZ1BdoWj)

![](https://drive.google.com/uc?export=download&id=1dMaxOaRX7wGqN4D_KMmZ1QK4yGt8hgG1)

在開啓之後，應該會被導入一下頁面，若顯示 **停用按鈕**，代表 API 已經被開啓，可以被程式操作了

![](https://drive.google.com/uc?export=download&id=1Hro_yFbGu04tTKFFF7zYiYqLjZfK1rt0)

接下來我們重複上述流程，將 Drive API 開啓

### 4. 在憑證的選項，選擇建立憑證 > 服務帳戶金鑰

![](https://drive.google.com/uc?export=download&id=1GiaFytTcsc4HBKSwfcyaVjgJo5rJXvfh)

![](https://drive.google.com/uc?export=download&id=1Y3YIshrJcUXTNOUd09fxSDXupc4BtfUu)

### 5. 服務帳戶金鑰，一個 JSON 檔案，會被下載到本機端

### 6. 將 JSON 檔案的路徑複製下來



## 測試一下我們的服務帳戶金鑰

In [0]:
import pygsheets
# 驗證
gc = pygsheets.authorize(service_file=r"C:\服務帳戶金鑰 json檔案路徑")
gc

## 範例 Google Spreadsheet
接下來請開啓並且複製一份這節課的範例 Google Spreadsheet 到你的 Google Drive：[連結](https://docs.google.com/spreadsheets/d/1CYXJyVGc9xcAM3lfMZtRHgOOK8jNACWPQWgxqELCW8o/edit#gid=0)

接下來請記得分享你的 Google Spreadsheet，並且調整權限至**編輯**

![](https://drive.google.com/uc?export=download&id=1Nd9avXDJvQjj8SNvSDO1YWzaCv7F_TVn)

In [0]:
wb = gc.open_by_url("你的 Google Spreadsheet url")
wb

In [0]:
# 檢查 url
wb.url

In [0]:
# 選擇追蹤清單
wks = wb.worksheet_by_title("追蹤清單")
wks

## 截取追蹤清單内的所有股票代號


```python
wks.get_col(1, include_tailing_empty=False)
```

In [0]:
wks.get_col(1, include_tailing_empty=False)

In [0]:
# 找出所有的股票代號，'TWSE_SEM_INDEX_1' 代表加權指數
stocks = wks.get_col(1, include_tailing_empty=False)[1:]
stocks

## 回顧我們之前實作的股票爬蟲

```python
import requests

def get_stock_data(stock_id, token):
    payload = {
        "symbolId": stock_id,
        "apiToken": token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/quote", params=payload)
    data = res.json()["data"]["quote"]

    return {
        "high": data["priceHigh"]["price"],
        "low": data["priceLow"]["price"],
        "open": data["priceOpen"]["price"],
        "trading": data["trade"]["price"]
    }
```

In [0]:
import requests

def get_stock_data(stock_id, token):
    payload = {
        "symbolId": stock_id,
        "apiToken": token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/quote", params=payload)
    data = res.json()["data"]["quote"]

    return {
        "high": data["priceHigh"]["price"],
        "low": data["priceLow"]["price"],
        "open": data["priceOpen"]["price"],
        "trading": data["trade"]["price"]
    }

## 接下來我們來測試一下寫好的爬蟲

請各位同學使用之前申請的 Fugle API Token 來測試

fugle 開發者頁面：[連節](https://developer.fugle.tw/)

In [0]:
# 測試一下是否能運作
get_stock_data("2330", "你的 Fugle API 通行碼")

In [0]:
# 截取多個股票的資料
fugle_token = "你的 Fugle API 通行碼"
stocks

for stock in stocks:
    print(get_stock_data(stock, fugle_token))

In [0]:
# 截取多個不同的工作表
for stock in stocks:
    wks = wb.worksheet_by_title(str(stock))
    print(wks)

In [0]:
for stock in stocks:
    wks = wb.worksheet_by_title(str(stock))
    col_a_data = wks.get_col(1, include_tailing_empty=False)
    # 動態偵測每一個工作表的最後一個 row
    max_row = len(col_a_data)
    print(max_row)

## 建立工作表

```python
wb.add_worksheet("Test", # 名稱
               rows=100, # row 數目
               cols=26,  # col 數目
               src_worksheet=wb.worksheet_by_title("範例"), # 複製格式的工作表 
               index=None)
```

In [0]:
wks = wb.add_worksheet("Test", 
                       rows=1000, 
                       cols=26,  
                       src_worksheet=wb.worksheet_by_title("範例"), 
                       index=None)
wks

## 產生一個新的 Row

```python
# row 代表被插入的 row 值, value 則是要寫入該 row 的資料
wks.insert_rows(row=1, values=[5,4,3,2,1])
```

In [0]:
wks.insert_rows(row=1, values=[5,4,3,2,1])

In [0]:
col_a_data = wks.get_col(1, include_tailing_empty=False)
last_row = len(col_a_data)
wks.insert_rows(row=last_row, values=[5,4,3,2,1])

In [0]:
fugle_token = "你的 Fugle API 通行碼"
stock = "2330"

stock_data = get_stock_data(stock, fugle_token)
stock_data

In [0]:
import time

data = [
    time.strftime("%Y/%m/%d"), 
    stock_data["open"], 
    stock_data["high"], 
    stock_data["low"], 
    stock_data["trading"]
]

data

## 整合一下目前的東西

In [0]:
fugle_token = "你的 Fugle API 通行碼"
stock = "2330"

stock_data = get_stock_data(stock, fugle_token)
stock_data

data = [
    time.strftime("%Y/%m/%d"), 
    stock_data["open"], 
    stock_data["high"], 
    stock_data["low"], 
    stock_data["trading"]
]

col_a_data = wks.get_col(1, include_tailing_empty=False)
last_row = len(col_a_data)
wks.insert_rows(last_row, values=data)

## 用 for loop 迭代股票代號，將資料寫入不同的工作表

In [0]:
fugle_token = "你的 Fugle API 通行碼"

for stock in stocks:
    # 選擇工作表
    wks = wb.worksheet_by_title(str(stock))
    print(wks)
    # 偵測該工作表最後一個 row
    col_a_data = wks.get_col(1, include_tailing_empty=False)
    last_row = len(col_a_data)
    # 截取該股票資料
    stock_data = get_stock_data(stock, fugle_token)
    data = [
        time.strftime("%Y/%m/%d"), 
        stock_data["open"], 
        stock_data["high"], 
        stock_data["low"], 
        stock_data["trading"]
    ]
    print(data)
    # 將資料寫入工作表
    wks.insert_rows(last_row, values=data)
    # 重整工作表
    wks.refresh()

In [0]:
fugle_token = "你的 Fugle API 通行碼"

for stock in stocks:
    # 選擇工作表
    try:
        wks = wb.worksheet_by_title(str(stock))
        col_a_data = wks.get_col(1, include_tailing_empty=False)
        # 偵測該工作表最後一個 row
        last_row = len(col_a_data)
    # 若該工作表不存在，就新建立一個
    except:
        wks = wb.add_worksheet(str(stock), rows=100, cols=26, src_tuple=None, src_worksheet=wb.worksheet_by_title("範例"), index=None)
        last_row = 1

    print(wks)
    # 截取該股票資料
    stock_data = get_stock_data(stock, fugle_token)
    data = [
        time.strftime("%Y/%m/%d"), 
        stock_data["open"], 
        stock_data["high"], 
        stock_data["low"], 
        stock_data["trading"]
    ]
    print(data)
    # 將資料寫入工作表
    wks.insert_rows(last_row, values=data)
    # 重整工作表
    wks.refresh()

## 完整版程式碼

在執行完畢後，試算表内所有的試算表都應該被填入資料

In [0]:
import pygsheets
import requests
import time

fugle_token = "你的 Fugle API 通行碼"
line_token = "你的 Line Notify 權杖"

def get_stock_data(stock_id, token):
    payload = {
        "symbolId": stock_id,
        "apiToken": token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/quote", params=payload)
    data = res.json()["data"]["quote"]

    return {
        "high": data["priceHigh"]["price"],
        "low": data["priceLow"]["price"],
        "open": data["priceOpen"]["price"],
        "trading": data["trade"]["price"],
        "rate": (data["trade"]["price"] - data["priceOpen"]["price"]) * 100 / data["priceOpen"]["price"]
    }

gc = pygsheets.authorize(service_file=r"C:\服務帳戶金鑰 json 檔案的路徑")
# 開啓範例試算表
wb = gc.open_by_url("你的 Google SpreadSheet 網址")
wks = wb.worksheet_by_title("追蹤清單")
# 截取股票清單
stocks = wks.get_col(1, include_tailing_empty=False)[1:]

for stock in stocks:
    try:
        # 選擇工作表
        wks = wb.worksheet_by_title(str(stock))
        # 偵測該工作表最後一個 row
        col_a_data = wks.get_col(1, include_tailing_empty=False)
        last_row = len(col_a_data)
    # 若無工作表，就建立
    except:
        wks = wb.add_worksheet(str(stock), rows=100, cols=26, src_tuple=None, src_worksheet=wb.worksheet_by_title("範例"), index=None)
        last_row = 1

    print(wks)
    # 截取該股票資料
    stock_data = get_stock_data(stock, fugle_token)
    data = [
        time.strftime("%Y/%m/%d"), 
        stock_data["open"], 
        stock_data["high"], 
        stock_data["low"], 
        stock_data["trading"]
    ]
    print(data)
    # 將資料寫入工作表
    wks.insert_rows(last_row, values=data)
    # 重整工作表
    wks.refresh()


# Line Notify
line_url = "https://notify-api.line.me/api/notify"

headers = {
    "Authorization": "Bearer " + line_token, 
    "Content-Type" : "application/x-www-form-urlencoded"
}

msg = f"BOT: {time.strftime('%Y/%m/%d')} 台股歷史資料已經更新完畢！"

payload = {'message': msg }
r = requests.post(line_url, headers = headers, params = payload)

## 將我們的程式碼部署到雲端上

搞定我們的程式碼之後，最後是要將程式碼部署到 Cloud Function 上，部署前有兩點要注意：

1. 我們需要將原本像脚本一樣的程式碼封裝到函式内（記得 Cloud Function 是執行使用者指定名稱的函數）
2. 由於 Cloud Function 沒有地方可以讓我們放置**服務帳戶金鑰**，因此我們需要將**服務帳戶金鑰**改變成環境參數(Environment Variable)

我們先將程式碼修改成以下，請注意關鍵是在這一行：

```python
gc = pygsheets.authorize(service_account_env_var="GOOGLE_CREDENTIALS")
```

Python 在執行時，會去搜尋執行環境内是否有名爲 **GOOGLE_CREDENTIALS** 的環境參數

In [0]:
import pygsheets
import requests
import time

fugle_token = "你的 Fugle API 通行碼"
line_token = "你的 Line Notify 權杖"

def get_stock_data(stock_id, token):
    payload = {
        "symbolId": stock_id,
        "apiToken": token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/quote", params=payload)
    data = res.json()["data"]["quote"]

    return {
        "high": data["priceHigh"]["price"],
        "low": data["priceLow"]["price"],
        "open": data["priceOpen"]["price"],
        "trading": data["trade"]["price"]
    }

def line_notify(msg, line_token):
    # Line Notify
    line_url = "https://notify-api.line.me/api/notify"

    headers = {
        "Authorization": "Bearer " + line_token, 
        "Content-Type" : "application/x-www-form-urlencoded"
    }

    payload = {'message': msg }
    r = requests.post(line_url, headers = headers, params = payload)   


def tw_stock_updater(request):
    # 驗證 Google Service Account
    gc = pygsheets.authorize(service_account_env_var="GOOGLE_CREDENTIALS")

    # 開啓範例試算表
    wb = gc.open_by_url("你的 Google SpreadSheet 網址")
    wks = wb.worksheet_by_title("追蹤清單")
    # 截取股票清單
    stocks = wks.get_col(1, include_tailing_empty=False)[1:]

    # 迭代每一支股票代號
    for stock in stocks:
        try:
            # 選擇工作表
            wks = wb.worksheet_by_title(str(stock))
            # 偵測該工作表最後一個 row
            col_a_data = wks.get_col(1, include_tailing_empty=False)
            last_row = len(col_a_data)
        # 若無工作表，就建立
        except:
            wks = wb.add_worksheet(str(stock), rows=100, cols=26, src_tuple=None, src_worksheet=wb.worksheet_by_title("範例"), index=None)
            last_row = 1

        print(wks)
        # 截取該股票的交易資料
        stock_data = get_stock_data(stock, fugle_token)
        # 將資料寫入一個 1 x 5 的串列
        data = [
            time.strftime("%Y/%m/%d"), 
            stock_data["open"], 
            stock_data["high"], 
            stock_data["low"], 
            stock_data["trading"]
        ]
        print(data)
        # 新增一個 row，將上面裝滿資料的串列寫入該 row
        wks.insert_rows(last_row, values=data)
        # 重整工作表，顯示更新之後的結果
        wks.refresh()

    # 產生訊息，提醒使用者股票資料已經更新完畢
    msg = f"BOT: {time.strftime('%Y/%m/%d')} 台股歷史資料已經更新完畢！"
    # 發送 Line 訊息
    line_notify(msg, line_token)

    return



### 1. 接下來我們就建立一個新的 Cloud Function

![](https://drive.google.com/uc?export=download&id=13JuK8lrWU8bf_8KOy5aw7Jk797tbTqGC)

### 2. 將我們的程式碼複製到編輯器
![](https://drive.google.com/uc?export=download&id=1u2tgbDdGpw1jVd9lI7MvUMQgprGnlb88)

### 3. 點選編輯器旁的 `requirement.txt`，填入 `pygsheets`

*這邊是在告訴 Cloud Function，在執行我們的 Python 程式碼之前需要安裝的模組

![](https://drive.google.com/uc?export=download&id=1_lALkcP5yNqAxhvJoIpojp7ycKK17lLm)

### 4. 接下來最有挑戰的部分，便是設定環境參數

注意環境參數的形式不能是一個檔案，必須是數字或是字串，因此，我們需要將**服務帳戶金鑰** 的 `json` 檔案轉成字串，最後再將該字串填入環境參數的值

![](https://drive.google.com/uc?export=download&id=1-N8pZ0vVyc2Kya5kfqS-32Et2SeICzWG)



In [0]:
import json

# 開啓服務帳戶金鑰 json檔
with open(r"C:\服務帳戶金鑰 json檔案路徑") as json_file:
    data = json.load(json_file)

# 將服務帳戶金鑰 json 檔轉換成 string
json_str = json.dumps(data)

# 但是因爲 Python 編碼的問題，可能會導致字串内多出 \ 跳脫字元，需要刪除掉
lst = json_str.split("\\n")
"\n".join(lst)

# 最後，請複製這段程式碼執行的結果（記得去掉前後端的單引號 '），並且填入 

## 如何定時執行我們的函式

Google Cloud Function 也具備了類似排程的服務：Cloud Scheduler

### 1. 開啓 Cloud Scheduler 服務
![](https://drive.google.com/uc?export=download&id=1vFB3Tb9AavHr1wcfjFh6DWyNpJkmeN6m)
![](https://drive.google.com/uc?export=download&id=170B57ka6UlqvNzSRNq_W_P8K5BvruAhP)
![](https://drive.google.com/uc?export=download&id=1Pe87n6c02SCyhikrkuJqND3EkkRzddis)

### 2. 選擇地點
![](https://drive.google.com/uc?export=download&id=1cFc2dfJQdIk5_bhXzeFbfrYRPqafOkhj)
![](https://drive.google.com/uc?export=download&id=1_7ZLqyq4MN26pa1xY_hBH3tch7_MhGFX)

### 3. 設定 Cloud Scheduler 工作

這邊比較麻煩的是設定 Unix 系統的排程，這邊提供一個好用的工具給大家：[Crontab guru](https://crontab.guru/#*_*_*_*)

![](https://drive.google.com/uc?export=download&id=1T2xNKkbmmKsIofEGQ17_F-md0bWofTI3)

### 4. 最後將我們的工作設定成

![](https://drive.google.com/uc?export=download&id=1C5aZaUbUsmjP3Z6mIHdWnfeNquJpQP5u)

### 5. 工作建立完畢後，可以測試執行是否能運作

![](https://drive.google.com/uc?export=download&id=1r_2Jl9dr2jVEF62GihfXY839AmQOW7yz)

### 6. 大公告成！

接下來就等著每個交易日讓 Cloud Scheduler 自動執行，並且將股價資料整理到 Google Spreadsheet 内

Cloud Scheduler 價格：[官方文件](https://cloud.google.com/scheduler/pricing)

# pyxl 課程總結

1. **寫程式是手段，而非目的**，若今天花費過多時間寫程式，而沒有把本業的工作做好，反而得不償失

2. 學習一個**能夠做更多事的程式語言，從時間報酬率的角度看是非常划算的投資**

3. 用程式解決問題，記住 Occam's Razar (奧卡姆剃刀)，也就是**“如無必要，誤增實體”**

4. 一個系統需要，前端，後端，以及 DataStore，而 **Excel 本身就是一個簡單的界面與 DataStore**，結合**用 Python 寫的後端**，就可以讓你快速客制化出一個驗證想法的程式 \ 系統

5. 從結構的角度來看，**Excel 與 Google Spreadsheet 是一樣的**，學會了其中一者，另外一者就能夠快速上手

6. 我們可以善用雲服務平台，像是 GCP，讓我們的程式能夠運行在雲端上，達到真正的自動化
